# Using TigerGraph CoPilot - InquiryAI

This notebook demostrates how to use InquiryAI in TigerGraph CoPilot, an AI assistant for your TigerGraph database.

## Prerequisites

First of all, a running TigerGraph database with version 3.9+ is required. The easist way to get a database running is through [TigerGraph Cloud](https://tgcloud.io). To run TigerGraph on your own environment, please refer to the [official doc](https://docs.tigergraph.com/tigergraph-server/current/getting-started/) for details.

Second, we assume you already have the CoPilot service set up. If not, there are two ways to set it up currently. Later, CoPilot will be available on TigerGraph Cloud as a service.  
1. Please follow the [instructions](https://github.com/tigergraph/CoPilot?tab=readme-ov-file#getting-started) there to set it up. You will download the source code of CoPilot and run it as a docker container. This way you will be able to modify and extend CoPilot yourself.  
2. Or you can pull and run the docker image built by TigerGraph as follows:

    ```bash
    docker run -d -v ./configs/llm_config.json:/llm_config.json -v ./configs/db_config.json:/db_config.json --name copilot -p 80:80 tigergraphml/copilot
    ```
    
    <br>where `llm_config.json` and `db_config.json` are configuration files for your LLM provider and TigerGraph database. Their structures are specified [here](https://github.com/tigergraph/CoPilot?tab=readme-ov-file#create-llm-provider-configuration-file).
    
Last, we will use Python package `pyTigerGraph` in this notebook to interact with CoPilot. (You can also communicate with the http endpoints of CoPilot directly if you are not using Python.) As CoPilot is under active development, please use the latest version of pyTigerGraph. To install or update pyTigerGraph, simply
```python
pip install -U pyTigerGraph
```

## Create Connections

In [1]:
from pyTigerGraph import TigerGraphConnection

In [2]:
# We first create a connection to the database
conn = TigerGraphConnection(
    host="https://webinar.i.tgcloud.io",
    username="user_1", 
    password="MyPassword1!")

# And then add CoPilot's address to the connection. This address 
# is the host's address where the CoPilot container is running.
conn.ai.configureInquiryAIHost("http://18.222.202.55")

## Ingest Data

If there is already a graph in your TigerGraph database against which you would like to query, then please feel free to skip this section. However, you will probably need to modify the questions below for your own data. 

Otherwise, we will download and ingest an example dataset, the digital infrastructure graph, which contains TBA

In [3]:
from pyTigerGraph.datasets import Datasets

# First download the dataset
dataset = Datasets("DigitalInfra")

A folder with name DigitalInfra already exists in ./tmp. Skip downloading.


In [4]:
# Then ingest it into the database. If your database doesn't have token
# authentication enabled, then set `getToken` to False. Databases on 
# TigerGraph Cloud always have token authentication enabled.
conn.ingestDataset(dataset, getToken=True)

---- Checking database ----
---- Creating graph ----
The graph DigitalInfra is created.
---- Creating schema ----
Using graph 'DigitalInfra'
Successfully created schema change jobs: [create_digital_infra_graph].
Kick off schema change job create_digital_infra_graph
Doing schema change on graph 'DigitalInfra' (current version: 0)
Trying to add local vertex 'Microservice' to the graph 'DigitalInfra'.
Trying to add local vertex 'Container' to the graph 'DigitalInfra'.
Trying to add local vertex 'BareMetalNode' to the graph 'DigitalInfra'.
Trying to add local vertex 'NodeMetric' to the graph 'DigitalInfra'.
Trying to add local vertex 'ContainerMetric' to the graph 'DigitalInfra'.
Trying to add local vertex 'UserService' to the graph 'DigitalInfra'.
Trying to add local vertex 'Trace' to the graph 'DigitalInfra'.
Trying to add local vertex 'MSCall' to the graph 'DigitalInfra'.
Trying to add local edge 'RUNS_IN' and its reverse edge 'reverse_RUNS_IN' to the graph 'DigitalInfra'.
Trying to add

## Query

In [7]:
# Now we can ask ours questions 
conn.ai.query("How many servers are there?")

{'natural_language_response': 'There are 46148 servers.',
 'answered_question': True,
 'query_sources': {'function_call': "getVertexCount('BareMetalNode')",
  'result': 46148,
  'reasoning': "The function getVertexCount is used because we need to count the number of vertices of a specific type, in this case 'BareMetalNode'. The function returns the number of vertices of the specified type, which is exactly what the question is asking for."}}